# Supplementary material for:
### Effective number of breeders from sibship reconstruction: empirical evaluations using hatchery steelhead

In [1]:
import numpy as np
from __future__ import division
import itertools
import collections

## Parentage analysis without parents (PWOP)
Waples et. al. (2011) equation 2a, also Supplemental equation 2 in the current manuscipt.

Produces an Ne estimate from a vector of k_i values using the equation:

\begin{equation}
 N_e = \frac{\sum_{}^{} k_i - 1}{\frac{\sum_{}^{} k_i^2}{\sum_{}^{} k_i} -1 } 
\end{equation}

In [2]:
def pwop(k_i):
    assert type(k_i) is np.ndarray
    print '{} contributing parents'.format(len(k_i))
    print '{} gametes'.format(np.sum(k_i))
    numerator = np.sum(k_i)-1
    denominator = np.sum(k_i**2)/np.sum(k_i) - 1
    Ne = numerator/denominator 
    return(Ne)

#### example

In [3]:
# K_i is an array giving the number of offspring produced by each parent.
K_i = np.array([2,2,3,7,2,5,1])
print('The PWOP Ne estimate is: {:1.3f}'.format(pwop(K_i)))

7 contributing parents
22 gametes
The PWOP Ne estimate is: 6.243


## p_same
calculates the probability that two gametes sampled at random from a k_i vector share the same parent. N is the number of parents.
\begin{equation}
p_{same} = \frac{1}{N(N-1)} \sum_{i=1}^{N} (k_i(k_i-1))
\end{equation}

In [4]:
def p_same(k_i):
    """chance that two random gametes present in the offspring generation are from the same parent"""
    N = np.int(np.sum(k_i)) # number of offspring or parents
    print '{} contributing parents'.format(len(k_i))
    print '{} gametes'.format(N)
    p = 1./(N*(N-1)) * np.sum(k_i*(k_i-1))
    return(p)

#### example

In [5]:
# use the same k_i vector
print(K_i)
print('The chance two gametes share a parent: {:1.3f}'.format(p_same(K_i)))

[2 2 3 7 2 5 1]
7 contributing parents
22 gametes
The chance two gametes share a parent: 0.160


## COLONY
"Equations for the effective size ($N_e$) of a population were derived in terms of the frequencies of a pair of offspring taken at random from the population being sibs sharing the same one or two parents" Wang (2009) eq. 10:

\begin{equation}
\frac{1}{N_e} = \frac{1+3\alpha}{4}(Q_1+Q_2+2Q_3)-\frac{\alpha}{2}(\frac{1}{N_1}+\frac{1}{N_2})
\end{equation}

Where $\alpha$ is a measure of departure from Hardy-Weinberg equilibrium, $Q_1$, $Q_2$, and $Q_3$ are the probabilities of a pair of offspring being paternal half-siblings, maternal half-siblings, and full-siblings, respectively. $N_1$ and $N_2$ are the number of male and female parents.

If we assume alpha = 0 (i.e. complete HWE). The equation simplifies to:

\begin{equation}
\frac{1}{N_e} = \frac{1}{4}(Q_1+Q_2+2Q_3)
\end{equation}

(Q1+Q2+2*Q3)/2 is eqivalent to p_same (Wang 2009, equation 8), so we can again simplify to:
\begin{equation}
\frac{1}{N_e} = \frac{1}{2} * p_{same}
\end{equation}

The chance two gametes that share a parent are identical by decent is 0.5, so we again simplyfy:

\begin{equation}
\frac{1}{N_e} = p(IBD)
\end{equation}

This is essentially the definition of inbreeding Ne.

But notice the impact of the simplifications, we lose all consideration of separate sexes and assuming alpha = 0, may not be reasonable, hat Wang (2009) states that α = 1/(1 − 2Ne) in a randomly mating population.

In [6]:
def COLONY(p_same_parent):
    """Given the chance that two random gametes share a parent, return the Ne"""
    Ne = 1./p_same_parent
    return(Ne)

In [7]:
print('The COLONY Ne estimate is: {:1.3f}'.format(COLONY(p_same(K_i))))

7 contributing parents
22 gametes
The COLONY Ne estimate is: 6.243


## Simulate an ideal mating population
Generates a K_i vector from one generation of random mating of N parents producing N offspring (total of 2N gametes). The result excludes parents that do not contribute offspring. 

In [8]:
def sim_ideal(N):
    """Generates a K_i vector from one generation of random mating of N parents"""
    a = collections.defaultdict(int)
    for xx in np.random.randint(0, N, 2*N): # For each of 2N gametes select a parent
        a[xx]+=1 # assign the gamete to that parent
    k_i = np.array(a.values())
    print '{} contributing parents'.format(len(k_i))
    print '{} gametes'.format(np.sum(k_i))
    return(k_i)

In [9]:
K_i = sim_ideal(200)
print (K_i)

176 contributing parents
400 gametes
[1 1 1 1 2 6 1 2 3 2 2 6 1 3 1 3 2 2 2 1 2 2 6 2 1 3 3 4 1 1 1 4 4 1 2 2 4
 3 1 3 5 2 1 3 1 1 1 1 1 4 2 1 2 3 2 2 4 1 3 2 1 1 2 3 1 2 1 3 7 4 3 1 8 2
 2 3 1 2 2 2 6 4 3 1 2 3 3 2 5 4 4 3 1 1 2 1 1 1 3 1 1 1 3 2 1 3 3 1 1 2 3
 3 1 2 3 1 3 4 2 1 3 2 3 1 3 3 1 1 1 2 3 2 3 2 1 2 3 3 5 3 1 1 3 2 3 1 1 3
 4 2 1 2 1 2 5 2 2 2 3 2 4 1 4 1 1 2 2 3 2 3 1 1 1 2 3 1]


## Apply both methods to the simulated k_i vector

In [10]:
p_same(K_i)

176 contributing parents
400 gametes


0.005112781954887218

In [11]:
pwop(K_i)

176 contributing parents
400 gametes


195.58823529411765

In [12]:
COLONY(p_same(K_i))

176 contributing parents
400 gametes


195.58823529411765

## Also compare to Crow (1954) & Crow and Denniston (1988) eq 1:
\begin{equation}
\frac{1}{N_e} = \frac{ \bar{k_i} - 1 + \frac{Var(k_i)}{\bar{k_i}}  }
{N \bar{k_i} -1}
\end{equation}

In [13]:
def Crow_Denniston(k_i):
    top = np.mean(k_i) - 1 + np.var(k_i)/np.mean(k_i)
    bottom = len(k_i)*np.mean(k_i)-1
    Ne = bottom/top
    return(Ne)

In [14]:
Crow_Denniston(K_i)

195.58823529411768